<a href="https://colab.research.google.com/github/marijnwolf/marijnwolf/blob/master/Week3_Marijn_Wolf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Fisher Matrix Code

First of all i will define all the variables and functions given by paper arXiv:gr-qc/04111464v4. Then I will define all the functions that are needed to compute the derivative of the waveform function h. 


In [1]:
###############import necassary stuff####################################
!pip install numpy
!pip install sympy
from math import *
from numpy import *
from sympy import *
from scipy import *
import matplotlib.pyplot as plt
from scipy.special import *
from scipy.integrate import *

#################### Defining variables ########################



## masses of black holes (arbatrary)
M_1 = 10**40            #mass of first object
M_2 = 10**41            #mass of second object
M_tot = M_1 + M_2       # total mass

## cutoff frequencies of Ligo detector (given in paper)
f_upper = pow((pow(6,(3/2))*pi*M_tot), -1) # higher cutoff frequency, Hz
f_lower = 40.0 # lower cutoff frequency, Hz

#### Natural constants
c = 2.998*10**8    # speed of light  (m/s)
G = 6.67*10**-11    # gravitational constant (m^3/kg/s^2 )



t_c =  1     #time of collision (value in this case is not important)
phi_c = 1    #phase             (value in this case is not important)
eta = (M_1 * M_2)/pow(M_tot, 2) # the dimensionless mass ratio
Mchirp = eta**(3/5)*M_tot     # kg
def v (M_tot, f):
 pow((pi*M_tot*f), (1/3)) # velocity


## alphas: not all are used later because would only result in a bit more accurary and a lot more complexity 
a_0 = 1.0
a_1 = 0.0
a_2 = (20/9) * ((743/336) + (11/4)*eta)
a_3 = -16*pi
a_4 = 10*((3058673/1016064) + (5429/1008)*eta +(617/144)*pow(eta, 2) )


SNR = 10.0              #a measure of how overall strong the gravitational wave is compared to the noise

## Defining for Power Spectral Density S(f) for Ligo
f_0 = 150               #(a scaling frequency given by paper (Hz)
S_0 = 9 * pow(10, -46)  # Hz




In [0]:
############################# Defining functions
from scipy import integrate

def x (f, f_0):                 
  return f/f_0
##print (x(column_1, f_0)) (just for testing purposes)

def psi (f, t_c, phi_c, eta, M_tot ):  # unitless  (phase radians)
 return (2 * pi * f * t_c - phi_c - (pi/4) + (3/(128 * eta * pow(v(M_tot, f), 5))) + 1) 
##print (psi (10000, t_c, phi_c, eta, M_tot )) (just for testing purposes)

def S (S_0, x):                 #The initial LIGO noise curve
 return S_0*((4.49*x)**-56 + (0.16*x)**-4.52 + 0.52 + 0.32*x*x)
## print (S(S_0,5)) (just for testing purposes)


def A (SNR, f, S, S_0, x, f_lower, f_upper):  # Defining A as given in document
 integral = integrate.quad(lambda f: (f**(-7/6)/S(S_0, x)), f_lower, f_upper)
 return sqrt(SNR/(4*integral)) 
## print(A(SNR, 100, S, S_0, x, f_lower, f_upper))  (just for testing purposes)

def h (A, psi):                 # Fourier domain waveform
  return A * pow(f, (-7/6)) * pow(math.e, (math.i * psi))


def hlong (A, M_tot, eta, phi_c, t_c) :  # Same waveform formula but extende so that parameters now correspond to var for later derivation
  return A * pow(f, (-7/6)) * pow(math.e, (math.i * ((2 * math.pi * f * t_c - phi_c - (math.pi/4) + (3/(128 * eta * pow(v(M_tot, f), 5))) + 1)))) 

In [3]:
##############################   Fisher Matrix


#### unfortunatly i am not sure why the sympy way of derivation gives so many errors
## I was not able to figure out this crucial step.....

from scipy import integrate
import sympy as sympy ## idk but otherwise sympy didnt work...



  ##### First define the parameters on which the waveform function is going to be derived in an array
var = [t_c, phi_c, eta, M_tot]


###defining partial derivatives (with help of sympy package)

t_c, phi_c, eta, M_tot = sympy.symbols('t_c, phic_c, eta, M_tot') ##define variables as symbols all at once

PartDeriv_t_c = sympy.diff(hlong, t_c)
PartDeriv_phi_c = sympy.diff(hlong, phi_c)
PartDeriv_eta = sympy.diff(hlong, eta)
PartDeriv_M_tot = sympy.diff(hlong, M_tot)

PartDeriv_arr = [PartDeriv_t_c, PartDeriv_phi_c, PartDeriv_eta, PartDeriv_M_tot]

##defining numerator of fisher formula, which is a 4x4 array
diffs = PartDeriv_arr * PartDeriv_arr

###  fisher matrix outcome (does not work unfortunatly)
matrixf = []
def fisher ():
    matrixf = 4 * (integrate.quad(lambda f:((diffs)/S), f_upper, f_lower ))
    print(matrixf)
    print (type(matrixf))
    return matrixf

fisher()


############## Invert Fisher matrix (does not work because troubles earlier, but this is how i would do it if working)
from numpy.linalg import inv
invert = inv(np.matrix(matrixf))


############## take diagonal elements of Fisher matrix
diagonals = np.diagonal(matrixf)
sqrdiag = np.sqrt(diagonals) ## square root of diagonals


SympifyError: ignored

### Exercise 2
  Generalising the code

### Exercise 3
Testing the generalized code


In [0]:

#################import data from text files (please move files in appropriate directory)###################

import numpy as np

########LIGO

Ligodata = np.loadtxt(fname = "LigoList.txt")
column_1 = Ligodata[:,0]   ### takes first colomn (frequencies)
column_6 = Ligodata[:,-1]  ### Always takes last column (sensitivies)

freq_ligo = column_1
sens_ligo = column_6

###########EINSTEIN

Einsteindata = np.loadtxt(fname = "EinsteinList.txt")
column_1 = Einsteindata[:,0]   ### takes first colomn (frequencies)
column_last = Einsteindata[:,-1]  ### Always takes last column (sensitivies)

freq_einstein = column_1
sens_einstein = column_last


############VIRGO

Virgodata = np.loadtxt(fname = "VirgoList.txt")
column_1 = Virgodata[:,0]   ### takes first colomn (frequencies)
column_last = Virgodata[:,-1]  ### Always takes last column (sensitivies)

freq_virgo = column_1
sens_virgo = column_last



############KAGRA

Kagradata = np.loadtxt(fname = "KagraList.txt")
column_1 = Kagradata[:,0]   ### takes first colomn (frequencies)
column_last = Kagradata[:,-1]  ### Always takes last column (sensitivies)

freq_kagra = column_1
sens_kagra = column_last



############## Invert Fisher matrix (does not work because troubles earlier, but this is how i would do it if working)
from numpy.linalg import inv
invert = inv(np.matrix(matrixf))


############## take diagonal elements of Fisher matrix
diagonals = np.diagonal(matrixf)
sqrdiag = np.sqrt(diagonals) ## square root of diagonals

